In [1]:
from __future__ import absolute_import, division, print_function

import numpy as np
import cv2
import os
from tqdm import tqdm
from random import shuffle
import matplotlib.pyplot as plt
import tensorflow as tf
import time
from tensorflow.python.client import device_lib

In [2]:
#hyperparams
LR=1e-3
batch_size=32
epochs=4


#Variables
TRAIN_DIR='/home/rahulchakwate/My_tensorflow/DataSets/cats_and_dogs_2/train'
IMG_SIZE=50
nb_classes=2


#Placeholders
images_ph=tf.placeholder(tf.float32, shape=[None,IMG_SIZE,IMG_SIZE,1])
labels_ph=tf.placeholder(tf.float32, shape=[None,nb_classes])

#save model path
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name='checkpoints\\'
if not os.path.isdir(save_dir):
     os.makedirs(save_dir)
model_save_name = os.path.join(save_dir, model_name)

In [3]:

def create_label(image_name):
    word_label=image_name.split('.')[-3]
    if word_label=='cat':
        return np.array([1,0])
    elif word_label=='dog':
        return np.array([0,1])


In [4]:
def create_train_data():
    training_data=[]
    for img in tqdm(os.listdir(TRAIN_DIR)):
        path=os.path.join(TRAIN_DIR,img)
        img_data=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img_data=cv2.resize(img_data,(IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img_data),create_label(img)])
    shuffle(training_data)
    np.save('train_data.npy',training_data)
    return training_data



In [5]:
#train_data=create_train_data()
train_data=np.load('/home/rahulchakwate/My_tensorflow/train_data.npy')

In [6]:
train=train_data
X_train=np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
y_train=np.array([i[1] for i in train])

images=X_train[0:2000]
val_images=X_train[2000:3000]
labels=y_train[0:2000]
val_labels=y_train[2000:3000]

total_batch=images.shape[0]
val_batch_size=val_images.shape[0]
print(images.shape)
print(val_images.shape)
print(labels.shape)
print(val_labels.shape)
print(total_batch)
print(val_batch_size)

(2000, 50, 50, 1)
(1000, 50, 50, 1)
(2000, 2)
(1000, 2)
2000
1000


In [7]:
def batch_dispatch(batch_size, batch ,images ,labels):
    batch_images=images[batch_size*(batch):batch_size*(batch+1)]
    batch_labels=labels[batch_size*(batch):batch_size*(batch+1)]
    return batch_images, batch_labels

In [8]:
def cnn_model(images_ph,nb_classes):
    #input_layer=tf.reshape(features["x"],[-1,28,28,1])
    conv1=tf.layers.conv2d(images_ph, 32,[5,5],padding="same",activation=tf.nn.relu)
    pool1=tf.layers.max_pooling2d(conv1,[2,2],strides=2)
    conv2=tf.layers.conv2d(pool1,64,[3,3],padding="same",activation=tf.nn.relu)
    pool2=tf.layers.max_pooling2d(conv2,[2,2],strides=2)
    flat1=tf.layers.flatten(pool2)
    dense1=tf.layers.dense(flat1,1024,activation=tf.nn.relu)
    drop1=tf.layers.dropout(dense1,0.5)
    logits=tf.layers.dense(drop1,nb_classes,activation=None)
    print(logits.shape)
    return logits

In [9]:
def trainacc(logits, labels):
  """Evaluate the quality of the logits at predicting the label.
  Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size], with values in the
      range [0, NUM_CLASSES).
  Returns:
    A scalar int32 tensor with the number of examples (out of batch_size)
    that were predicted correctly.
  """
  with tf.variable_scope('train_accuracy') as scope:
      correct = tf.equal(tf.argmax(logits,1), tf.argmax(labels,1))
      correct = tf.cast(correct, tf.float32)
      accuracy = tf.reduce_mean(correct)
      print(correct.shape)
      tf.summary.scalar(scope.name+'/train_accuracy', accuracy)
      return accuracy

In [10]:
def valacc(logits, labels):
  """Evaluate the quality of the logits at predicting the label.
  Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size], with values in the
      range [0, NUM_CLASSES).
  Returns:
    A scalar int32 tensor with the number of examples (out of batch_size)
    that were predicted correctly.
  """
  with tf.variable_scope('val_accuracy') as scope:
      correct = tf.equal(tf.argmax(logits,1), tf.argmax(labels,1))
      correct = tf.cast(correct, tf.float32)
      accuracy = tf.reduce_mean(correct)
      print(correct.shape)
      tf.summary.scalar(scope.name+'/val_accuracy', accuracy)
      return accuracy

In [11]:
def trainer(logits, total_batch, batch_size):
    sess=tf.Session()
    cross_entropy=tf.nn.softmax_cross_entropy_with_logits(labels=labels_ph,logits=logits)
    cost=tf.reduce_mean(cross_entropy)
    val_cost=tf.reduce_mean(cross_entropy)
    tf.summary.scalar("cost",cost)
    tf.summary.scalar("val_cost",val_cost)
    optimizer=tf.train.AdamOptimizer(learning_rate=LR).minimize(cost)
    trainaccuracy = trainacc(logits,labels_ph)
    valaccuracy = valacc(logits, labels_ph)
    sess.run(tf.global_variables_initializer())
    writer=tf.summary.FileWriter(model_save_name,graph=tf.get_default_graph())
    merged=tf.summary.merge_all()
    saver=tf.train.Saver(max_to_keep=4)
    counter=0
    for epoch in range(epochs):
        for batch in range(int(total_batch / batch_size)-1):
            counter+=1
            batch_images, batch_labels= batch_dispatch(batch_size, batch, images, labels)
            

            loss, summary=sess.run([cost, merged], feed_dict={images_ph: batch_images, labels_ph: batch_labels})
            train_acc=sess.run([trainaccuracy], feed_dict={images_ph: batch_images, labels_ph: batch_labels})

            #print('loss',loss)
            sess.run(optimizer, feed_dict={images_ph: batch_images, labels_ph: batch_labels})
            if batch % 10 == 0:
                print('epoch no',epoch, 'batch no',batch,'loss',loss,'train_acc',train_acc)
            writer.add_summary(summary,counter)
        val_loss,summary=sess.run([val_cost,merged],feed_dict={images_ph: val_images, labels_ph: val_labels})
        val_acc=sess.run([valaccuracy],feed_dict={images_ph: val_images, labels_ph: val_labels})
        writer.add_summary(summary,counter)

        print('epoch_no','epoch','val_loss',val_loss,'val_acc',val_acc)
        saver.save(sess, model_save_name)
        print('Saved trained model at %s ' % model_save_name)


In [12]:
logits=cnn_model(images_ph,nb_classes)
trainer(logits,total_batch, batch_size)

(?, 2)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

(?,)
(?,)
epoch no 0 batch no 0 loss 13.848972 train_acc [0.40625]
epoch no 0 batch no 10 loss 2.7531643 train_acc [0.5]
epoch no 0 batch no 20 loss 0.7676078 train_acc [0.5625]
epoch no 0 batch no 30 loss 0.70857286 train_acc [0.40625]
epoch no 0 batch no 40 loss 0.8039432 train_acc [0.40625]
epoch no 0 batch no 50 loss 0.6513021 train_acc [0.59375]
epoch no 0 batch no 60 loss 0.7291782 train_acc [0.5625]
epoch_no epoch val_loss 0.6850221 val_acc [0.574]
Saved trained model at /home/rahulchakwate/My_tensorflow/saved_models/checkpoints\ 
epoch no 1 batch no 0 loss 0.6581572 train_acc [0.65625]
epoch no 1 batch no 10 loss 0.6625564 train_acc [0.5]
epoch no 1 batch no 20 loss 0.6565533 train_acc [0.53125]
epoch no 1 batch no 30 loss 0.5957391 train_acc [0.59375]
epoch no 1 batch no 40 loss 0.858